# SINDy From Scratch

The goal of this file is to implement the algorithm from scratch, applied to the Lorenz system to reproduce the result of the paper. The optimization algorithm is sequential thresholding with threshold parameter $\lambda$.

In [1]:
import numpy as np
from scipy.integrate import solve_ivp

# Generate Data

In [2]:
# generate data

def lorenz(t, x, sigma=10, beta=8/3, rho=28):
    """
    Lorenz system data

    Args:
        x (array): Data
        sigma (float, optional): Parameter. Defaults to 10.
        beta (float, optional): Parameter. Defaults to 8/3.
        rho (float, optional): Parameter. Defaults to 28.

    Returns:
        array: Lorenz system data
    """
    return [sigma * (x[1] - x[0]), x[0] * (rho - x[2]) - x[1], x[0] * x[1] - beta * x[2]]

x0 = [-8, 7, 27]
delta_t= 0.001
t = np.arange(0, 100, delta_t)
x = solve_ivp(lorenz, t_span = (t[0], t[-1]), y0 = x0, t_eval=t, rtol = 1e-12, method = "LSODA", atol = 1e-12).y.T

In [3]:
x[:5]

array([[-8.        ,  7.        , 27.        ],
       [-7.85082366,  6.98457872, 26.87275344],
       [-7.70328918,  6.9683473 , 26.74700466],
       [-7.55738803,  6.95135327, 26.62273958],
       [-7.41311133,  6.93364275, 26.49994362]])

In [4]:
# derivative 
x_dot = np.array([lorenz(0, x[t]) for t in range(t.size)]) # X'
x_dot[:5]

array([[ 150.        ,  -15.        , -128.        ],
       [ 148.35402387,  -15.83439269, -126.49537176],
       [ 146.71636478,  -16.62053277, -125.0045401 ],
       [ 145.08741298,  -17.35984468, -123.52804619],
       [ 143.46754079,  -18.05372769, -122.06638194]])

# Functions for the Algorithm

In [5]:
def SparseRegression(library, derivative, n, alpha = 0.1, max_iter = 20):
    """
    Computes sparse optimization via least-squares using sequential thresholding

    Args:
        library (array_like): A matrix representing the library used to model the dynamics
        derivative (array_like): A matrix representing the time derivatives of the state variable x
        n (int): The dimensionality of the state variable x
        alpha (float, optional): Threshold parameter to cut-off coefficients to 0. Defaults to 0.1.
        max_iter (int, optional): Number of sequential least-squares to compute. Defaults to 20.

    Returns:
        coeff (ndarray): Coefficients xi
    """
    
    if alpha < 0 or n < 0 or max_iter < 0:
        raise ValueError("Negative input non accepted")
    
    coeff = np.linalg.lstsq(library, derivative, rcond=None)[0] # least-square, no sparse
    
    for i in range(max_iter): # remove small coefficients
        small_coeff_bool = np.abs(coeff) < alpha # 0 if >=lambd, 1 if < lambd
        coeff[small_coeff_bool] = 0
        for k in range(n): # iterate on each state dimension
            big_coeff_bool = small_coeff_bool[:, 0] == 0 # column of current dimension that has non-small coefficients
            coeff[big_coeff_bool, k] = np.linalg.lstsq(library[:, big_coeff_bool], derivative[:, k], rcond=None)[0] # update non-small coeff on new least-squares 
    
    return coeff

In [6]:
def generatePolyLibrary(x_in, n, poly_order=3):
    """
    Generates the library matrix from data, with polynomial (very inefficiently, but it works). The paper works with poly_order 3.

    Args:
        x_in (array_like): Input data
        n (int): Number of variables in the input data x
        poly_order (int): Maximum polynomial order for feature generation. Defaults to 3.
        
    Returns:
        theta: Expanend data according to library matrix
    """
    
    m = x_in.shape[0]
    p = 1 + n + (n * (n + 1) // 2) + (n * (n + 1) * (n + 2) // (2 * 3)) + 11 # found in the code of the library (3rd order poly)
    theta = np.zeros((m,p)) # library initialized as empty
    
    # order 0 (constants)
    column_p = 0
    theta[:, column_p] = np.ones(m)
    
    # order 1 (linear)
    column_p = 1
    for i in range(n):
        theta[:, column_p] = x_in[:, i]
        column_p += 1
        
    if poly_order >= 2: # order 2 (quadratic)
        for i in range(n):
            for j in range(i, n):
                theta[:, column_p] = x_in[:, i] * x_in[:, j] # i and j columns of input
                column_p += 1
    if poly_order >= 3: # order 3 (cubic)
        for i in range(n):
            for j in range(i, n):
                for k in range(j, n):
                    theta[:, column_p] = x_in[:, i] * x_in[:, j] * x_in[:, k]
                    column_p += 1
    if poly_order >= 4:
        for i in range(n):
            for j in range(i, n):
                for k in range(j, n):
                    for l in range(k, n):
                        theta[:, column_p] = x_in[:, i] * x_in[:, j] * x_in[:, k] * x_in[:, l]
                        column_p += 1
    if poly_order >= 5:
        for i in range(n):
            for j in range(i, n):
                for k in range(j, n):
                    for l in range(k, n):
                        for m in range(l, n):
                            theta[:, column_p] = x_in[:, i] * x_in[:, j] * x_in[:, k] * x_in[:, l] * x_in[:, m]
                            column_p += 1
    
    return theta

# Algorithm

In [12]:
poly_lib = generatePolyLibrary(x_in=x, n=x.shape[1], poly_order=3) # thetha
coefficients = SparseRegression(library=poly_lib, derivative=x_dot, n=x.shape[1], alpha=0.05, max_iter=20)

In [13]:
coefficients

array([[  0.        ,   0.        ,   0.        ],
       [-10.        , -12.90889359,   0.05654449],
       [ 10.        ,   9.93951837,  -0.06973613],
       [  0.        ,   0.        ,  -2.66666667],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   1.        ],
       [  0.        ,  -1.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.     